In [ ]:
# imports
import warnings, numpy as np, pandas as pd
from pathlib import Path
warnings.filterwarnings("ignore")

# this is the seed that was the best according to our first attempt in the first notebook
SEED = 27
rng = np.random.default_rng(SEED)

# more imports
from sklearn.model_selection import (
    GroupShuffleSplit, GroupKFold, RandomizedSearchCV, cross_val_predict
)
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.metrics import (
    f1_score, balanced_accuracy_score, roc_auc_score, average_precision_score,
    precision_recall_curve
)
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.base import clone
from sklearn.calibration import CalibratedClassifierCV

# short cuts for file paths
DATA_DIR = Path("../data")
EEG_FILE = DATA_DIR / "clean_eeg.csv"
SUMMARY_FILE = DATA_DIR / "challenger_insight_session_summary.csv"

# different bands 
BASE_BANDS = ["alpha_power","beta_power","theta_power","gamma_power"]


In [ ]:
# Function to calculate the metrics 
def metrics_report(y_true, proba, thr=0.5, name="Results"):
    y_pred = (proba >= thr).astype(int)
    return pd.DataFrame([{
        "Model": name,
        "F1": round(f1_score(y_true, y_pred), 3),
        "BalancedAcc": round(balanced_accuracy_score(y_true, y_pred), 3),
        "ROC-AUC": round(roc_auc_score(y_true, proba), 3),
        "PR-AUC": round(average_precision_score(y_true, proba), 3),
        "Thr": round(thr, 3),
    }])

# Function to choose the threshold for the max F1 score
def choose_threshold_max_f1(y_true, proba):
    p, r, t = precision_recall_curve(y_true, proba)
    if len(t) == 0: return 0.5
    f1 = (2*p*r)/np.clip(p+r, 1e-9, None)
    return float(t[np.nanargmax(f1[:-1])])


# Function to log the experiments
experiment_log = []
def log_experiment(name, y_true, proba, thr):
    df = metrics_report(y_true, proba, thr, name)
    experiment_log.append(df.iloc[0].to_dict())
    return df

# function to show the experiments
def show_experiments(sort_by="F1"):
    if not experiment_log:
        return print("No experiments logged.")
    df = pd.DataFrame(experiment_log).sort_values(sort_by, ascending=False).reset_index(drop=True)
    display(df)
    print(f"🏁 Best by {sort_by}: {df.iloc[0]['Model']} ({df.iloc[0][sort_by]:.3f})")
    return df


In [ ]:
# loading the files 
eeg = pd.read_csv(EEG_FILE)
summary = pd.read_csv(SUMMARY_FILE)

# cleaning the columns for consistency purposes
eeg.columns = eeg.columns.str.strip().str.lower()
summary.columns = summary.columns.str.strip().str.lower()

# getting the median of the mean_disengage_risk column and using that as the threshold
thr_risk = summary["mean_disengage_risk"].median()
summary["disengaged"] = (summary["mean_disengage_risk"] > thr_risk).astype(int)

# merging the labels with the eeg data
df = eeg.merge(summary[["subject_id","session_id","disengaged"]],
               on=["subject_id","session_id"], how="left", validate="many_to_one")
df = df.dropna(subset=["disengaged"]).copy()
df["disengaged"] = df["disengaged"].astype(int)

# ensuring that the segments are in the correct order
if "segment_idx" not in df.columns:
    df = df.sort_values(["session_id","timestamp"]).reset_index(drop=True)
    df["segment_idx"] = df.groupby("session_id").cumcount()

# creating new features
df["beta_alpha"]  = df["beta_power"]  / (df["alpha_power"] + 1e-6)
df["theta_alpha"] = df["theta_power"] / (df["alpha_power"] + 1e-6)
total = df[BASE_BANDS].sum(axis=1).replace(0, 1e-6)
df["alpha_rel"] = df["alpha_power"]/total
df["beta_rel"]  = df["beta_power"]/total
df["theta_rel"] = df["theta_power"]/total
df["gamma_rel"] = df["gamma_power"]/total

# creating the features and target
FEATURES = BASE_BANDS + ["beta_alpha","theta_alpha","alpha_rel","beta_rel","theta_rel","gamma_rel"]
TARGET = "disengaged"
GROUPS = "session_id"

print("Segments:", df.shape, "| class ratio (1s):", round(df[TARGET].mean(),3))


Segments: (4126, 15) | class ratio (1s): 0.503


In [ ]:
# splitting the data into training and testing sets
# using the GroupShuffleSplit to split the data into training and testing sets
# ensures no session_id appears in both train and test.
gss = GroupShuffleSplit(n_splits=1, test_size=0.2, random_state=SEED)
groups_all = df[GROUPS]
train_idx, test_idx = next(gss.split(df[FEATURES], df[TARGET], groups_all))

X_tr = df.loc[train_idx, FEATURES].reset_index(drop=True)
X_te = df.loc[test_idx,  FEATURES].reset_index(drop=True)
y_tr = df.loc[train_idx, TARGET].reset_index(drop=True)
y_te = df.loc[test_idx,  TARGET].reset_index(drop=True)
g_tr = groups_all.iloc[train_idx].reset_index(drop=True)
g_te = groups_all.iloc[test_idx].reset_index(drop=True)

# printing the ratio of 1s in the training and testing sets
print("Train 1-ratio:", round(y_tr.mean(),3), "| Test 1-ratio:", round(y_te.mean(),3))


Train 1-ratio: 0.498 | Test 1-ratio: 0.52


In [ ]:
# import
from sklearn.preprocessing import StandardScaler

# aggregate per session (mean/std/median/min/max for each feature)
agg_funcs = {f: ["mean","std","median","min","max"] for f in FEATURES}
sess = df.groupby(["subject_id","session_id"], as_index=False).agg(agg_funcs)
sess.columns = ["subject_id","session_id"] + ["_".join(c) for c in sess.columns[2:]]
sess = sess.merge(df[["session_id","disengaged"]].drop_duplicates(), on="session_id", how="left")

# creating X (features) and y (target) from the aggregated session data
X_sess = sess.drop(columns=["disengaged","subject_id","session_id"])
y_sess = sess["disengaged"].astype(int)
groups_sess = sess["subject_id"]  # cross-subject grouping (can use session_id if you prefer)

# splitting the data into training and testing sets
gss2 = GroupShuffleSplit(n_splits=1, test_size=0.2, random_state=SEED)
tr_s, te_s = next(gss2.split(X_sess, y_sess, groups_sess))
X_tr_s, X_te_s = X_sess.iloc[tr_s], X_sess.iloc[te_s]
y_tr_s, y_te_s = y_sess.iloc[tr_s], y_sess.iloc[te_s]
g_tr_s = groups_sess.iloc[tr_s]

# creating a pipeline for the session-level logistic regression
pipe_sess = Pipeline([
    ("sc", StandardScaler()),
    ("lr", LogisticRegression(max_iter=5000, class_weight="balanced", random_state=SEED))
])

space_sess = {"lr__C": np.logspace(-2,1,8), "lr__solver":["lbfgs","liblinear"]}
cv_sess = GroupKFold(5)

# performing a randomized search for the best hyperparameters
rs_sess = RandomizedSearchCV(pipe_sess, space_sess, n_iter=12, scoring="f1",
                             cv=cv_sess, random_state=SEED, n_jobs=-1, refit=True)
rs_sess.fit(X_tr_s, y_tr_s, groups=g_tr_s)

# OOF threshold selection
oof_sess = cross_val_predict(rs_sess.best_estimator_, X_tr_s, y_tr_s, groups=g_tr_s,
                             cv=cv_sess, method="predict_proba", n_jobs=-1)[:,1]
thr_sess = choose_threshold_max_f1(y_tr_s.values, oof_sess)

# test
proba_te_sess = rs_sess.best_estimator_.fit(X_tr_s, y_tr_s).predict_proba(X_te_s)[:,1]
display(log_experiment("Session LR", y_te_s.values, proba_te_sess, thr_sess))


,Model,F1,BalancedAcc,ROC-AUC,PR-AUC,Thr
0,Session LR,0.737,0.75,0.714,0.576,0.431


In [ ]:
# merging session-level metadata (stimulus type, modality, etc.) into the EEG dataframe”
safe_cols = ["session_id","stimulus_type","task_difficulty","modality","n_segments"]
ctx = summary[safe_cols].drop_duplicates("session_id")
df_ctx = df.merge(ctx, on="session_id", how="left")

# defining which columns are numeric and which are categorical for preprocessing
num_cols = FEATURES + ["n_segments"]
cat_cols = ["stimulus_type","task_difficulty","modality"]

pre = ColumnTransformer([
    ("num", "passthrough", num_cols),
    ("cat", OneHotEncoder(handle_unknown="ignore"), cat_cols),
])

# build pipeline that encodes data then trains a HistGradientBoostingClassifier (HGB) on session context features
pipe_ctx = Pipeline([("prep", pre), ("clf", HistGradientBoostingClassifier(random_state=SEED))])

# preparing training and testing sets for the context-level model
X_tr_ctx = df_ctx.loc[train_idx, num_cols + cat_cols].reset_index(drop=True)
X_te_ctx = df_ctx.loc[test_idx,  num_cols + cat_cols].reset_index(drop=True)

# define parameter search space for randomized hyperparameter tuning
space_ctx = {
    "clf__learning_rate": [0.02, 0.05, 0.1],
    "clf__max_leaf_nodes": [15, 31, 63],
    "clf__max_depth": [None, 3, 5, 8],
    "clf__min_samples_leaf": [10, 20, 30],
    "clf__l2_regularization": [0.0, 0.1, 0.5, 1.0],
}
cv_ctx = GroupKFold(5)
rs_ctx = RandomizedSearchCV(pipe_ctx, space_ctx, n_iter=20, scoring="f1",
                            cv=cv_ctx, random_state=SEED, n_jobs=-1, refit=True)
rs_ctx.fit(X_tr_ctx, y_tr, groups=g_tr)

# manual group-aware calibration (robust)
cal_models, oof_cal = [], np.zeros(len(X_tr_ctx))
for tr_i, va_i in cv_ctx.split(X_tr_ctx, y_tr, g_tr):
    Xt, Xv = X_tr_ctx.iloc[tr_i], X_tr_ctx.iloc[va_i]
    yt, yv = y_tr.iloc[tr_i], y_tr.iloc[va_i]
    est = clone(rs_ctx.best_estimator_).fit(Xt, yt)
    method = "isotonic" if yv.nunique()==2 else "sigmoid"
    cal = CalibratedClassifierCV(est, method=method, cv="prefit").fit(Xv, yv)
    oof_cal[va_i] = cal.predict_proba(Xv)[:,1]
    cal_models.append(cal)

# threshold (max F1) — or swap for a precision policy if you want fewer false positives
thr_ctx = choose_threshold_max_f1(y_tr.values, oof_cal)
proba_ctx_te = np.mean([m.predict_proba(X_te_ctx)[:,1] for m in cal_models], axis=0)

display(log_experiment("HGB+Ctx (Manual-Cal)", y_te.values, proba_ctx_te, thr_ctx))


,Model,F1,BalancedAcc,ROC-AUC,PR-AUC,Thr
0,HGB+Ctx (Manual-Cal),0.542,0.648,0.659,0.763,0.486


In [ ]:
# final report
show_experiments("F1")      # sort by F1
show_experiments("PR-AUC")  # also view probability quality


,Model,F1,BalancedAcc,ROC-AUC,PR-AUC,Thr
0,Session LR,0.737,0.750,0.714,0.576,0.431
1,HGB+Ctx (Manual-Cal),0.542,0.648,0.659,0.763,0.486


🏁 Best by F1: Session LR (0.737)


,Model,F1,BalancedAcc,ROC-AUC,PR-AUC,Thr
0,HGB+Ctx (Manual-Cal),0.542,0.648,0.659,0.763,0.486
1,Session LR,0.737,0.750,0.714,0.576,0.431


🏁 Best by PR-AUC: HGB+Ctx (Manual-Cal) (0.763)


,Model,F1,BalancedAcc,ROC-AUC,PR-AUC,Thr
0,HGB+Ctx (Manual-Cal),0.542,0.648,0.659,0.763,0.486
1,Session LR,0.737,0.750,0.714,0.576,0.431


In [1]:
# import 
from sklearn.metrics import confusion_matrix

# function to show the confusion matrix and explain the metrics
def show_confusion_and_explain(model_name, y_true, proba, thr):
    y_pred = (proba >= thr).astype(int)
    cm = confusion_matrix(y_true, y_pred, labels=[1,0])
    df_cm = pd.DataFrame(cm,
        index=["Actual 1 (disengaged)","Actual 0 (engaged)"],
        columns=["Pred 1 (disengaged)","Pred 0 (engaged)"])
    display(df_cm)
    TP, FN = cm[0,0], cm[0,1]; FP, TN = cm[1,0], cm[1,1]
    prec = TP / (TP + FP + 1e-9); rec = TP / (TP + FN + 1e-9)
    print(f"{model_name} → Precision={prec:.3f}, Recall={rec:.3f}")
